<a href="https://colab.research.google.com/github/jjcruzado/Pytorch-Tutorial/blob/main/simple_neuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# iris data set
import torch
import torch.nn as nn
import torch.nn.functional as F

In [41]:
# create a model class that inhertis the nn.module
# fc1 = fully connected 1 = hidden layer 1 (bcse its fully connected)
# relu = rectified linear unit = do something, if the output is less then zero, call it zero, otherwise use the output (think active vs. inactive)
class Model(nn.Module):
  # input layer (4 features of the flower) -->
  # hidden layer1 (some number of neurons) -->
  # hidden layer2 (some number of neurons) --> (
  # output (3 classes of iris flowers)
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__() # instantiate our nn.Module
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.out(x))

    return x


In [42]:
# pick a random seed for randomization
# neural network include randomization. in order to get us to match the video, put a seed

torch.manual_seed(41)

# create an instance of our model

model = Model()